In [1]:
import os
import gc
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error

import matplotlib.pyplot as plt

np.random.seed(42)

In [2]:
# 表示形式
pd.options.display.float_format = '{:.5f}'.format
np.set_printoptions(precision=5, floatmode='maxprec')
np.set_printoptions(suppress=True)

In [4]:
pressure_oof_df = pd.read_csv('../notebooks/results/1031_lstm_hidden=1024_ver11_mod-feat11_dropout=0.0/oof_df.csv')

In [5]:
masked_df = pressure_oof_df[['u_out', 'breath_id', 'R', 'C', 'pressure', 'preds']]

masked_df['error'] = masked_df['pressure']-masked_df['preds']
masked_df['error'] = masked_df.mask(masked_df['u_out']==1)['error']
masked_df['abs_error'] = np.abs(masked_df['pressure']-masked_df['preds'])
masked_df['abs_error'] = masked_df.mask(masked_df['u_out']==1)['abs_error']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [16]:
masked_df.head()

,u_out,breath_id,R,C,pressure,preds,error,abs_error
0,0,5,-0.33333,0.75000,6.32961,6.26317,0.06644,0.06644
1,0,5,-0.33333,0.75000,7.38414,7.32517,0.05897,0.05897
2,0,5,-0.33333,0.75000,8.36837,8.34564,0.02273,0.02273
3,0,5,-0.33333,0.75000,9.00109,9.01221,-0.01112,0.01112
4,0,5,-0.33333,0.75000,9.77441,9.76667,0.00774,0.00774


In [17]:
masked_df.describe()

,u_out,breath_id,R,C,pressure,preds,error,abs_error
count,6036000.00000,6036000.00000,6036000.00000,6036000.00000,6036000.00000,6036000.00000,2290968.00000,2290968.00000
mean,0.62045,62838.85870,0.15636,0.15202,11.22041,19.23815,-0.00415,0.17167
std,0.48528,36335.25619,0.43546,0.42881,8.10970,8.39123,0.38241,0.34173
min,0.00000,1.00000,-0.33333,-0.25000,-1.89574,-1.84352,-15.62096,0.00000
25%,0.00000,31377.00000,-0.33333,-0.25000,6.32961,12.45088,-0.08387,0.03342
50%,1.00000,62765.50000,0.00000,0.00000,7.03263,18.44033,-0.00192,0.08249
75%,1.00000,94301.00000,0.66667,0.75000,13.64103,25.34739,0.08108,0.18472
max,1.00000,125749.00000,0.66667,0.75000,64.82099,64.98778,51.79968,51.79968


In [12]:
for i in [-0.25, 0, 0.75]:
    for j in [-0.33, 0, 0.67]:
        print(f'C:{i} R:{j}')
        display(masked_df[(masked_df['C']==i) & (masked_df['R'].round(2)==j)].describe())

C:-0.25 R:-0.33


,u_out,breath_id,R,C,pressure,preds,error
count,664960.00000,664960.00000,664960.00000,664960.00000,664960.00000,664960.00000,249386.00000
mean,0.62496,62801.83578,-0.33333,-0.25000,11.70301,21.17658,-0.00566
std,0.48413,36351.05648,0.00000,0.00000,9.22148,8.71414,0.38066
min,0.00000,39.00000,-0.33333,-0.25000,3.30661,3.37432,-15.62096
25%,0.00000,31551.50000,-0.33333,-0.25000,6.04840,14.31569,-0.08288
50%,1.00000,62731.00000,-0.33333,-0.25000,6.47021,20.60244,-0.00245
75%,1.00000,94436.00000,-0.33333,-0.25000,14.97677,28.17438,0.08108
max,1.00000,125733.00000,-0.33333,-0.25000,56.59564,54.08908,5.45931


C:-0.25 R:0


,u_out,breath_id,R,C,pressure,preds,error
count,485600.00000,485600.00000,485600.00000,485600.00000,485600.00000,485600.00000,184106.00000
mean,0.62087,62461.87117,0.00000,-0.25000,11.99172,21.34041,-0.00189
std,0.48517,36104.37109,0.00000,0.00000,9.21796,8.36510,0.33010
min,0.00000,21.00000,0.00000,-0.25000,3.58782,3.70951,-4.80435
25%,0.00000,31432.00000,0.00000,-0.25000,6.11870,14.85832,-0.08504
50%,1.00000,61905.50000,0.00000,-0.25000,6.54051,21.03538,-0.00254
75%,1.00000,93610.00000,0.00000,-0.25000,15.39858,28.21048,0.08341
max,1.00000,125743.00000,0.00000,-0.25000,50.83086,50.83394,4.64076


C:-0.25 R:0.67


...

,u_out,breath_id,R,C,pressure,preds,error
count,661680.00000,661680.00000,661680.00000,661680.00000,661680.00000,661680.00000,245700.00000
mean,0.62867,62315.04582,-0.33333,0.75000,10.19783,17.38763,-0.00514
std,0.48316,36371.34385,0.00000,0.00000,6.08720,7.71012,0.17964
min,0.00000,5.00000,-0.33333,0.75000,4.29084,4.81054,-3.07252
25%,0.00000,30990.00000,-0.33333,0.75000,6.04840,10.55723,-0.07612
50%,1.00000,61856.00000,-0.33333,0.75000,7.59505,15.65089,-0.00091
75%,1.00000,94094.00000,-0.33333,0.75000,12.09438,23.31597,0.07292
max,1.00000,125735.00000,-0.33333,0.75000,40.35585,41.16803,1.60769


C:0.75 R:0


,u_out,breath_id,R,C,pressure,preds,error
count,654880.00000,654880.00000,654880.00000,654880.00000,654880.00000,654880.00000,243184.00000
mean,0.62866,63711.94405,0.00000,0.75000,10.74399,18.03742,0.00014
std,0.48316,36585.07402,0.00000,0.00000,7.06892,7.59377,0.25430
min,0.00000,1.00000,0.00000,0.75000,4.29084,4.49343,-2.66894
25%,0.00000,31880.00000,0.00000,0.75000,6.47021,11.55848,-0.09026
50%,1.00000,64386.50000,0.00000,0.75000,7.24353,17.12311,-0.00147
75%,1.00000,95395.00000,0.00000,0.75000,12.44589,23.96818,0.08935
max,1.00000,125722.00000,0.00000,0.75000,46.19092,47.17649,4.21180


C:0.75 R:0.67


,u_out,breath_id,R,C,pressure,preds,error
count,655120.00000,655120.00000,655120.00000,655120.00000,655120.00000,655120.00000,253113.00000
mean,0.61364,63530.70729,0.66667,0.75000,11.33802,17.28382,-0.01186
std,0.48692,36406.63463,0.00000,0.00000,8.05471,7.54378,0.52286
min,0.00000,4.00000,0.66667,0.75000,4.92356,4.97231,-5.23991
25%,0.00000,32317.00000,0.66667,0.75000,6.61082,11.43801,-0.10571
50%,1.00000,63839.00000,0.66667,0.75000,7.03263,16.05680,-0.00238
75%,1.00000,95330.00000,0.66667,0.75000,13.43012,22.06699,0.10129
max,1.00000,125745.00000,0.66667,0.75000,62.29011,61.92635,6.15262


# predsシフト

In [47]:
shift_df = masked_df
shift_df['preds'] -= 0.005

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [48]:
# shift_df = masked_df
# shift_df['preds'] = shift_df.mask(shift_df['R'].round(2)==-0.33 and shift_df['C']==0.75 , shift_df['preds']-0.002)

In [49]:
shift_df['error'] = shift_df['pressure']-shift_df['preds']
shift_df['error'] = shift_df.mask(shift_df['u_out']==1)['error']
shift_df['abs_error'] = np.abs(shift_df['pressure']-shift_df['preds'])
shift_df['abs_error'] = shift_df.mask(shift_df['u_out']==1)['abs_error']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [50]:
shift_df.describe()

,u_out,breath_id,R,C,pressure,preds,error,abs_error
count,6036000.00000,6036000.00000,6036000.00000,6036000.00000,6036000.00000,6036000.00000,2290968.00000,2290968.00000
mean,0.62045,62838.85870,0.15636,0.15202,11.22041,19.23337,0.00063,0.17169
std,0.48528,36335.25619,0.43546,0.42881,8.10970,8.39118,0.38241,0.34170
min,0.00000,1.00000,-0.33333,-0.25000,-1.89574,-1.84852,-15.61596,0.00000
25%,0.00000,31377.00000,-0.33333,-0.25000,6.32961,12.44610,-0.07911,0.03341
50%,1.00000,62765.50000,0.00000,0.00000,7.03263,18.43556,0.00287,0.08250
75%,1.00000,94301.00000,0.66667,0.75000,13.64103,25.34260,0.08584,0.18479
max,1.00000,125749.00000,0.66667,0.75000,64.82099,64.98278,51.80468,51.80468


In [31]:
shift_df[(shift_df['R'].round(2)==-0.33) & (shift_df['C']==0.75)]['preds']

0          6.26317
1          7.32517
2          8.34564
3          9.01221
4          9.76667
            ...   
6035755   16.56376
6035756   16.67217
6035757   16.65914
6035758   16.37208
6035759   15.40394
Name: preds, Length: 661680, dtype: float64

In [30]:
shift_df[(shift_df['R'].round(2)==-0.33) & (shift_df['C']==0.75)]['preds'].add(0.002)

0          6.26517
1          7.32716
2          8.34764
3          9.01421
4          9.76867
            ...   
6035755   16.56576
6035756   16.67417
6035757   16.66114
6035758   16.37408
6035759   15.40594
Name: preds, Length: 661680, dtype: float64

In [25]:
shift_df[(shift_df['R'].round(2)==-0.33) & (shift_df['C']==0.75)]['preds']+0.02

0          6.28317
1          7.34516
2          8.36564
3          9.03221
4          9.78667
            ...   
6035755   16.58376
6035756   16.69217
6035757   16.67914
6035758   16.39208
6035759   15.42394
Name: preds, Length: 661680, dtype: float64